<img src="https://pmarote.net/tcc/icmc_logo.png"  width="192px" align="right">
<p style="text-align: center; font-size: 200%; font-weight: bold; margin-top: 10px;"><b>UNIVERSIDADE DE SÃO PAULO</b></a></p>
<p style="text-align: center; font-size: 150%; font-weight: bold; margin-top: 10px;">Instituto de Ciências Matemáticas e de Computação</a></p><br>

---
### <div style="text-align: center"><font color="navy">Uso de técnicas de processamento de linguagem natural:<br><br>Aplicação em Decisões do Contencioso Administrativo Fiscal na Secretaria da Fazenda do Estado de São Paulo.</font></div>
### <div style="text-align: left"><font color= #333>Paulo Ricardo dos Santos Olim Marote<br><br>Trabalho de Conclusão de Curso - MBA em Ciência de Dados (CEMEAI)</font></div>
- **Parte II - Pesquisa das decisões por palavras**
<p style="text-align: right; font-size: 100%; opacity: 0.8; margin-top: 0px;">USP - São Carlos - 2023</p>

## 🚴 Carregamento de Bibliotecas Auxiliares e Bancos de Dados já criados e processados anteriormente no <i>notebook</i> 'tcc.ipynb'
- Antes de iniciar, é necessário baixar as bibliotecas necessárias, desenvolvidas para este TCC e gravar em /content/core
- Esta biblioteca encontra-se no github do autor, na pasta /core, no link https://github.com/pmarote/tcc
- Adicionalmente, são carregados em /content/var/ os arquivos .txt das decisões já criados e processados anteriormente no primeiro <i>notebook</i> de nome 'tcc.ipynb'

In [ ]:
import os

if not os.path.exists("/content/src/"):
    print('- Carregando o código fonte disponível no github para /content/src/')
    !mkdir -p /content/src/
    !git clone -q https://github.com/pmarote/tcc.git /content/src/tcc
else:
    print('- Código fonte de github já se encontra carregada em /content/src/')

if not os.path.exists("/content/core/"):
    print('- Criando pasta /content/core/')
    !mkdir -p /content/core/
    print('- Copiando pasta /content/src/tcc/core/ para /content/core/')
    !cp -r /content/src/tcc/core/ /content/
else:
    print('- Pasta /content/core/ já se encontra carregada')

if not os.path.exists("/content/var/"):
    print('\n- Criando a pasta /content/var/ para armazenar todos os dados')
    !mkdir -p /content/var/
else:
    print('- Pasta /content/var/ já se encontra criada')

if not os.path.exists("/content/var/txt/"):
    print('\n- Carregando os arquivos .txt já processados, em /content/var/txt/, de link público do google drive do autor')
    !gdown 1-47Pw0OFC43pTaS72ALvPW6VwtOieqCL
    print("- Arquivos de dados 'var_txt.zip' baixado com sucesso a partir de link público do google drive")
    print("- Descompactando o conteúdo para a pasta /content/var/txt/ . Aguarde a execução completa.")
    !unzip -q /content/var_txt.zip -d /content/var/
    !rm /content/var_txt.zip
    print('\n- Finalizado!')
else:
    print('- A pasta /content/var/ com os bancos de dados já criados e processados anteriormente já se encontra criada e carregada')

- Carregando o código fonte disponível no github para /content/src/
- Criando pasta /content/core/
- Copiando pasta /content/src/tcc/core/ para /content/core/

- Criando a pasta /content/var/ para armazenar todos os dados

- Carregando os arquivos .txt já processados, em /content/var/txt/, de link público do google drive do autor
Downloading...
From: https://drive.google.com/uc?id=1-47Pw0OFC43pTaS72ALvPW6VwtOieqCL
To: /content/var_txt.zip
100% 26.2M/26.2M [00:00<00:00, 69.3MB/s]
- Arquivos de dados 'var_txt.zip' baixado com sucesso a partir de link público do google drive
- Descompactando o conteúdo para a pasta /content/var/txt/ . Aguarde a execução completa.

- Finalizado!


## 🔛 Importações e Inicialização
  - Código inicial, importando as bibliotecas necessárias
  - Aqui também são definidas algumas funções 'helpers' para utilização nos códigos, como de formatação de números para português-Brasil

In [ ]:
import os
import ipywidgets as widgets
import pandas as pd
import numpy as np
from IPython.display import display, HTML

import core  # apenas insere o diretório ./core em sys.path
from Config import Config
from WebScraper import WebScraper
from Db import Db
config = Config.getConfigs()
db = Db(config['VAR_DIR'])
print("config = ", config)

# duas funções para dar uma melhor formatação em português
def int_f(val):
    return "{:,}".format(val).replace(',','.')
def two_dec_f(val):
    return "{:,.2f}".format(val).replace(',','#').replace('.',',').replace('#','.')
# styler em português para pandas
def df_style(styler):
    styler.format(lambda v: f'{v:,.2f}'.replace(".", "#").
              replace(",", ".").replace("#", ",") if isinstance(v, float) else v)
    styler.applymap(lambda v: 'color:red;' if not isinstance(v, str) and v < 0 else None)
    styler.applymap(lambda v: 'opacity: 80%;'
                    if not isinstance(v, str) and abs(v) < 10000 else None)
    return styler


Banco de dados /content/var/tcc.db inexistente... Criando...
config =  {'SRC_DIR': '/content', 'VAR_DIR': '/content/var', 'CWEB_DIR': '/content/var/www'}


## 👀 Pesquisa em bancos de dados de sentenças por palavras
- Utilização da ferramente de busca por texto chamada <i>Full Text Search (FTS5)</i>, disponibilizada da biblioteca de banco de dados <i>SQLite</i>


In [ ]:
# criação do arquivo sqlite chamado decisoes.db, com uma tabela chamada decisao, chave primaria 'id', campo 'nome', campo 'texto'
import sqlite3
conn = sqlite3.connect(os.path.join(config['VAR_DIR'], 'decisoes.db'))
c = conn.cursor()
c.execute('CREATE TABLE decisao (id INTEGER PRIMARY KEY, nome TEXT, texto TEXT)')
conn.commit()


In [ ]:
# carregando a tabela decisao com os dados extraídos da pasta txt, lendo todos os arquivos dessa pasta, carregando cada registro com id sequencial criado automaticamente, nome é o nome de cada arquivo lido e texto é o conteúdo de cada arquivo .txt lido
import re  # Importa o módulo de expressões regulares

for filename in os.listdir(os.path.join(config['VAR_DIR'], 'txt')):
    if filename.endswith('.txt'):
        with open(os.path.join(config['VAR_DIR'], 'txt', filename), 'r') as f:
            texto = f.read()
            # 'salvando' mudança de linha por <br>
            texto = texto.replace('\n', '<br>').replace('\r', '')
            # Substituindo todos os caracteres de controle ASCII por uma string vazia
            texto = re.sub('[\x00-\x1F]', '', texto)
            # 'recuperando' mudança de linha dos <br>
            texto = texto.replace('<br>', '\n')
        c.execute('INSERT INTO decisao (nome, texto) VALUES (?, ?)', (filename, texto))
conn.commit()


In [ ]:
# criando uma nova tabela sqlite do tipo full text search fts5, para que possa ser feita pesquisa por palavra e copie os dados da tabela decisao para essa tabela nova

conn = sqlite3.connect(os.path.join(config['VAR_DIR'], 'decisoes.db'))
c = conn.cursor()

c.execute('CREATE VIRTUAL TABLE decisao_fts5 USING FTS5(nome, texto)')
c.execute('INSERT INTO decisao_fts5 SELECT nome, texto FROM decisao')
conn.commit()


- Como cada sentença é bastante longa, é retornado apenas o trecho em que o termo pesquisado aparece

In [ ]:
def pesquisa_trechos_aiims(conn, query, tam_trecho: int, limite_linhas: int = -1):
    # Se não definido limite_linhas, retorna até 1000 linhas
    if limite_linhas == -1:
        limite_linhas = 1000
    c = conn.cursor()
    c.execute('SELECT DISTINCT nome, texto FROM decisao_fts5 WHERE texto MATCH ? LIMIT ?', (query, limite_linhas))
    rows = c.fetchall()
    conn.commit()
    result = {}
    for row in rows:
        nome, texto = row[0], row[1]
        index = texto.find(query)
        if index != -1:
            start = max(0, index - tam_trecho)  # Evitar índice negativo
            end = min(index + len(query) + tam_trecho*2, len(texto))  # Ajustar para no sobrepasar el total de caracteres del texto
            trecho = '(...)' + texto[start:end] + '(...)'
        else:
            trecho = f'Há o termo {query} neste AIIM, mas não consegui separá-lo'
        # Substituindo todos os caracteres de controle ASCII por um espaço
        trecho = re.sub('[\x00-\x1F]', ' ', trecho)
        # Eliminar caracteres estranhos, con valores Unicode maiores que 512
        trecho = re.sub('[\u0200-\U0010FFFF]', '', trecho)
        result[nome] = trecho
    return result

# Exemplo de uso
palavra_busca = 'insanável'
print(f"Alguns testes com buscando a seguinte palavra: {palavra_busca}")
resultado = pesquisa_trechos_aiims(conn, palavra_busca, tam_trecho=50, limite_linhas=3)
for nome, trecho in resultado.items():
    print(f"{nome}: {trecho}")

Alguns testes com buscando a seguinte palavra: insanável
4125696voto4824801.txt: (...) sua gênese, notas fiscais inidôneas contêm vício insanável que lhes retiram a qualidade finalística a que se destinavam. 48. Reitere-se que é irrelevante o fa(...)
4119239voto4880095.txt: (...)o AIIM apenas em relação a um dos Autuados. Vício insanável. Vício que não contamina o AIIM relativamente aos demais Autuados. Recurso de Ofício conhecido e pr(...)
4126281voto4823760.txt: (...)rado não pode prosseguir, uma vez que há um vício insanável. Na capitulação da multa, o AFR autuante considera a ocorrência do disposto no artigo 85, inciso II(...)


## 🔍 Pesquisa simples, por palavra
  - Execute para habilitar os widgets
  - Utilizando a função pesquisa_trechos_aiims(), automaticamente são retornados os trechos das sentenças em que aparece a palavra procuranda, além do link para consulta do AIIM e decisão completa no endereço eletrônico público do Tribunal de Impostos e Taxas (TIT)

In [ ]:
import re  # Importa o módulo de expressões regulares
conn = sqlite3.connect(os.path.join(config['VAR_DIR'], 'decisoes.db'))

from IPython.display import display, HTML, clear_output
# Criando os widgets
pesquisa = widgets.Text(value='', description='Digite o texto')
pesquisa_button = widgets.Button(description='Iniciar pesquisa')

# Definindo as ações dos botões
def on_pesquisa_button_clicked(b):
    clear_output()
    limite_linhas = 100
    result = pesquisa_trechos_aiims(conn, pesquisa.value, tam_trecho=100, limite_linhas=limite_linhas)
    msg_inicial = f"<h3>Quantidade de Ocorrências:{len(result)}</h3>"
    if len(result) >= limite_linhas:
        msg_inicial = f"<h3>(Abaixo mostrando as primeiras 100 ocorrências)</h3>"
    html = f"{msg_inicial}<h4>Texto pesquisado: {pesquisa.value}</h4><table><thead><tr><th>numero</th><th>sentença</th></tr></thead><tbody>"
    for key, value in result.items():
        html += '<tr><td  style="vertical-align: top">{}</td><td>{}</td></tr>'\
                .format(f'<a href="https://www.fazenda.sp.gov.br/epat/extratoprocesso/ExtratoDetalhe.aspx?num_aiim={key[0:7]}" target="_blank">Visualizar AIIM {key}</a>',\
                value)
    html += "</tbody></table>"
    display(HTML(html))

pesquisa_button.on_click(on_pesquisa_button_clicked)
display(pesquisa, pesquisa_button)

numero,sentença
Visualizar AIIM 4124057voto4824867.txt,"(...) do certificado digital ficasse restrito entre os seus funcionários e o escritório responsável pela contabilidade Plumas Assessoria Contábil (Plumas) (documento 09). - ingressou com o processo judicial nº. 1030960-75.2017.8.26.0114 (documento 10), com o objetivo de que fossem fornecidos os endereços de IP e (...)"
Visualizar AIIM 4129583voto4843858.txt,"(...)UTÁRIA DE JULGAMENTO DE BAURU UNIDADE DE JULGAMENTO DE SÃO JOSÉ DO RIO PRETO AIIM nº 4.129.583-3 contabilidade da empresa destinatária ou, ainda, outros documentos confeccionados pela própria compradora. Não se pode perder de vista que, em muitos casos como esses, de operações em que há documentos fiscais emi(...)"
Visualizar AIIM 4123872voto4840429.txt,"(...)litado de movimentar suas contas bancárias por dívidas trabalhistas e bancárias. 6.3. Informa que a contabilidade ADCONT deixou de efetuar as escritas fiscais por falta de pagamento dos serviços, sendo em seguida contratado o Sr. Arthur Sandrini Neto, mediante salário mensal de R$ 2.000,00, para emissão de notas(...)"
Visualizar AIIM 4126281voto4823760.txt,"(...)tal que constam na base de dados da Sefaz/SP) tendo em vista a inidoneidade do documento lançado na contabilidade da autuada. O documento supostamente emitido por Link Premium Imp. E Exportação EIRELI-EPP IE: 177.398.783.110 foi considerado inidôneo por não atender às condições estabelecidas no artigo 36, § 1º(...)"
Visualizar AIIM 4122482voto4820355.txt,"(...)mentos Ltda. em plena atividade; 4.9.5. em 1º de junho de 2015, o contador que presta serviços de contabilidade à Multigrãos Distribuidora de Alimentos Ltda., compareceu, espontaneamente, à repartição fiscal, prestou informações sobre a fiscalização que recaia sobre a Multigrãos Distribuidora de Alimentos Ltda(...)"
Visualizar AIIM 4124753voto4825761.txt,"(...)a mencionada na peça preambular da autuação; 16.10. Assim que identificou as inconsistências em sua contabilidade, realizou o estorno de todas as notas fiscais e estornou eventuais créditos de sua contabilidade, tendo em vista que a notas fiscais identificadas lançadas indevidamente contra sua pessoa não corresp(...)"
Visualizar AIIM 4123555voto4821472.txt,(...)CIA TRIBUTÁRIA DE JULGAMENTO DE SÃO PAULO AIIM nº 4.123.555-1 UNIDADE DE JULGAMENTO DE SÃO PAULO contabilidade deve espelhar os atos e fatos efetivamente praticados. Não se socorreu o contribuinte desse meio probatório; iii. O Item c Depreende-se dos autos que a Autuada não ofertou elementos competentes (...)
Visualizar AIIM 4127553voto4824810.txt,"(...)a a oportunidade de apresentar provas capazes de ilidir a acusação. Como é de conhecimento geral, a contabilidade deve espelhar os atos e fatos efetivamente ocorridos. Não se utilizou desse meio probatório a autuada para ilidir a acusação que lhe está sendo imposta pelo fisco através do presente AIIM. 8. Ressalt(...)"
Visualizar AIIM 4129096voto4828483.txt,"(...)ponsabilidade deste contribuinte, e não consta o frete em sua escrituração fiscal, nem mesmo em sua contabilidade tem o registro do Documento enviado mediante uso de assinatura eletrônica, de acordo com o caput do art. 75 da Lei 13.457, de 18/03/2009. GOVERNO DO ESTADO DE SÃO PAULO SECRETARIA DA FAZENDA E P(...)"
Visualizar AIIM 4126380voto4824700.txt,"(...)tratar das operações do último semestre de 2018. E também que estas operações eram uma questão da contabilidade e que seu contador, Sr. Enéas Ramos Leite Júnior, é que saberia como responder ao fisco. 14. A empresa GENERAL BENEFICIAMENTO foi notificada (Notificação n° IC/N/FIS/000001425/2019 fls.91/93) e ren(...)"


## 💡 Elaboração de teses com base nos excertos das decisões acima apresentadas
- A título exemplificativo, o resultado da célula acima foi lançado em um modelo GPT, com o seguinte prompt:
  - "Aqui temos alguns excertos de decisões de processos administrativos da Secretaria da Fazenda do Estado de São Paulo, com o termo 'contabilidade'. Quero que discorra como esse termo está sendo tratado nessas decisões: (...)"
- O resultado obtido foi o seguinte:

Os excertos das decisões de processos administrativos da Secretaria da Fazenda do Estado de São Paulo revelam diversos contextos nos quais o termo "contabilidade" é empregado. Aqui estão algumas observações gerais:

**1 - Contabilidade como Registro de Transações Comerciais e Fiscais:** Em várias decisões, a contabilidade é mencionada como um registro essencial das transações comerciais e fiscais das empresas. Há referências à necessidade de as empresas manterem registros contábeis precisos para cumprir suas obrigações fiscais.

**2 - Irregularidades e Inconsistências Contábeis:** Em alguns casos, as decisões destacam irregularidades ou inconsistências na contabilidade das empresas. Isso inclui a emissão de notas fiscais inidôneas, falhas na escrituração fiscal, e até mesmo a fraude contábil. Estes problemas são frequentemente a base para ações fiscais e penalidades.

**3 - Evidências e Provas em Disputas Fiscais:** A contabilidade das empresas é frequentemente utilizada como evidência em disputas fiscais. As decisões mencionam como a contabilidade pode ou não corroborar as alegações das empresas sobre suas operações e obrigações fiscais.

**4 - Impacto da Contabilidade na Tomada de Decisões Administrativas:** A precisão e integridade da contabilidade das empresas parecem ser cruciais para a tomada de decisões nos processos administrativos. Decisões de julgamento muitas vezes se baseiam na qualidade dos registros contábeis apresentados.

**5 - Responsabilidades dos Contadores:** Em várias instâncias, os contadores são mencionados em relação às suas responsabilidades na manutenção da contabilidade das empresas. Isso inclui casos em que contadores são contratados para corrigir problemas contábeis ou quando são responsabilizados por erros e omissões.

Essas observações indicam que a contabilidade é um elemento central e crítico nas decisões administrativas relacionadas a impostos e regulamentações fiscais, enfatizando a importância de práticas contábeis precisas e confiáveis.

## 🙋 Conclusões:
- Verifica-se, assim, também a possibilidade de análise de decisões com excertos das mesmas, através da busca por palavras, conforme apresentado neste trabalho, para elaboração de <i>prompts</i> complexos, com grande quantidade de detalhes, em ferramentas <i>GPT</i>
- Nesse sentido, conforme o exemplo desta célula, a possibilidade de identificação de padrões, tendências e inclusive possíveis inconsistências na aplicação da legislação tributária, contribuindo para aprimorar o sistema e promover maior eficiência e justiça fiscal
- A título exemplificativo, muito interessante que, com base na análise dos excertos, o modelo <i>GPT</i> tenha concluído algo muitas vezes esquecido, mas aqui bem destacado, que "<i>a precisão e integridade da contabilidade das empresas parecem ser cruciais para a tomada de decisões nos processos administrativos e que decisões de julgamento muitas vezes se baseiam na qualidade dos registros contábeis apresentados</i>"

## 🔠 Métodos para pesquisa avançada
  - Pesquisa por palavra, lematizando, retirando acentuações e <i>stop words</i>
  - a seguir, propostas para pesquisas avançadas, utilizando ferramentas de processamento de linguagem natural (PLN) através da biblioteca <i>nltk</i>
  - também foi utilizada a biblioteca <i>unidecode</i>, possibilitando a conversão de caracteres acentuados para não acentuados, de forma que a pesquisa se torne mais abrangente, independemente de critérios de acentuação
    - Referida medida foi efetuada em nova rodada de CRISP porque foram detectadas algumas inconsistências de digitação na origem, como por exemplo, 'crédito' e 'credito'

In [ ]:
%%capture
# unidecode é aqui usado basicamente para converter caracteres acentuados para não acentuados
# não está disponível nativamente no GoogleColab e, assim, na primeira utilização, necessário executar este código
!pip install unidecode
print('unidecode instalado com sucesso')

In [ ]:
import sqlite3
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import string
from unidecode import unidecode

# Configuração do NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))
stemmer = SnowballStemmer('portuguese')

# Função para processar o texto
def processa_texto(c_mem, text):
    # Remover pontuação e converter para minúsculas
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    # Tokenização e processamento
    words = text.split()
#    processed_words = [stemmer.stem(word) for word in words if word not in stop_words]
    processed_words = []
    for word in words:
        if word not in stop_words:
            processed_word = stemmer.stem(word)
            # Remover acentuação
            processed_word = unidecode(processed_word)
            processed_words.append(processed_word)
            # Inserir na tabela de palavras
            c_mem.execute('INSERT OR REPLACE INTO palavras VALUES (?, ?)', (word, processed_word))
#    for palavra, palavra_proc in zip(tokens_processados, tokens_processados_lemma):
#        c_mem.execute('INSERT OR REPLACE INTO palavras VALUES (?, ?)', (palavra, palavra_lemma))
    return ' '.join(processed_words)

# Conexão com o SQLite
conn = sqlite3.connect(os.path.join(config['VAR_DIR'], 'decisoes.db'))
cursor = conn.cursor()
cursor.execute('CREATE VIRTUAL TABLE IF NOT EXISTS decisao_fts5_proc USING FTS5(nome, texto)')
# reiniciando o banco de dados, caso já tenha preenchido antes
cursor.execute('DELETE FROM decisao_fts5_proc')

# Crie uma tabela em memória para armazenar as palavras e seus correspondentes processados
conn_mem = sqlite3.connect(':memory:')
c_mem = conn_mem.cursor()
c_mem.execute('CREATE TABLE palavras (palavra TEXT PRIMARY KEY, palavra_proc TEXT)')

# Ler os dados da tabela original e processá-los
cursor.execute('SELECT nome, texto FROM decisao')
rows = cursor.fetchall()

#contador = 0
for nome, texto in rows:
#    contador += 1
    # limite, para esta fase de testes, para ganhar tempo
#    if contador >= 200:
#        break
    texto_processado = processa_texto(c_mem, texto)
    # Inserir na tabela FTS5
    cursor.execute('INSERT INTO decisao_fts5_proc (nome, texto) VALUES (?, ?)', (nome, texto_processado))

conn.commit()
conn.close()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# prompt: lista de parte do conteúdo de decisao_fts5_proc, para visualização do resultado do processamento do unidecode e nltk

conn = sqlite3.connect(os.path.join(config['VAR_DIR'], 'decisoes.db'))
cursor = conn.cursor()

cursor.execute('SELECT count(nome) FROM decisao_fts5_proc')
row = cursor.fetchone()
print("Quantidade de decisões:", row[0])

cursor.execute('SELECT * FROM decisao_fts5_proc LIMIT 10')
rows = cursor.fetchall()
for row in rows:
  print(row)


Quantidade de decisões: 2538
('4123071voto4821463.txt', 'govern estad paul secret fazend planej delegac tributar julgament paul unidad julgament paul autu ie chong soo tak cpf aiim no 41230711 local paul 05235526880 solidari ie local cnpjcpf nihil fas julgament julgador nihil defes irin jacomet ement itcmd - deix pag impost omissa dev receb valor provenient doaco imov descrit declar itcmd nos 58501350 58501475 impost exig referes diferenc recolh autu calcul sefazsp term port cat no 152003 prejudic aprec merit - art 30 lei no 1345709 credit exigibil suspens mant mult jur mor aiim julg procedent pressupost conhec 1 conhec defes tempest acost aut mei advog constituid atend assim requisit admissibil previst artig 29 lei 13457 18 marc 2009 cc artig 94 decret 54486 26 junh 2009 dispo sobr process administr tributari decorrent lancament ofici da outr provident relatori 2 15042019 signatari inicial autu sob acus seguint irregular relat itcmd 21 deix pag itcmd montant r 658410 seis mil quinhent

In [ ]:
# teste: mostrando alguns registros da tabela palavras
c_mem.execute('SELECT palavra, palavra_proc FROM palavras LIMIT 1000, 20')
rows = c_mem.fetchall()
for row in rows:
    print(row)

('350358', '350358')
('575607', '575607')
('662670', '662670')
('737876', '737876')
('irradiaria', 'irradi')
('acobrança', 'acobr')
('10051008', '10051008')
('pellegrini', 'pellegrin')
('grinover', 'grinov')
('art104', 'art104')
('unilos', 'unil')
('dispensáveis', 'dispens')
('125226567', '125226567')
('sfpexp201917362', 'sfpexp201917362')
('322098600114', '322098600114')
('910913', '910913')
('nãolocalizada', 'naolocaliz')
('aramaki', 'aramak')
('paralisou', 'paralis')
('enriquecida', 'enriquec')


In [ ]:
pesquisa_teste = 'insanável'
pesq_processada = processa_texto(c_mem, pesquisa_teste)
print(f'Pesquisa: {pesquisa_teste}')
print(f'Pesquisa processada: {pesq_processada}')
c_mem.execute('SELECT palavra, palavra_proc FROM palavras WHERE palavra_proc = ? LIMIT 20', (pesq_processada,))
rows = c_mem.fetchall()
for row in rows:
    print(row)

Pesquisa: insanável
Pesquisa processada: insan
('insanáveis', 'insan')
('insanável', 'insan')


In [ ]:
def pesquisa_trechos_aiims_proc(conn, c_mem, query, tam_trecho: int, limite_linhas: int = -1):
    pesq_processada = processa_texto(c_mem, query)
    print(f'Pesquisa: {query}')
    print(f'Pesquisa processada: {pesq_processada}')
    print('Dessa pesquisa processada, serão retornadas as seguintes palavras:')
    c_mem.execute('SELECT palavra, palavra_proc FROM palavras WHERE palavra_proc = ? LIMIT 20', (pesq_processada,))
    rows = c_mem.fetchall()
    for row in rows:
        print(row)
    # Se não definido limite_linhas, retorna até 1000 linhas
    if limite_linhas == -1:
        limite_linhas = 1000
    c = conn.cursor()

    c.execute('SELECT DISTINCT nome FROM decisao_fts5_proc WHERE texto MATCH ?', (pesq_processada,))
    rows_1 = c.fetchall()
    conn.commit()

    # Extrair os nomes de rows_1 para uma lista simples
    nomes = [row[0] for row in rows_1]
    # Construir uma string de placeholders para a cláusula IN
    placeholders = ','.join('?' * len(nomes))
    print(f'nomes: {nomes}')
    print(f'placeholders: {placeholders}')

#    c.execute('SELECT DISTINCT nome, texto FROM decisao_fts5 WHERE nome IN (?) LIMIT ?', (rows_1, limite_linhas))
    c.execute(f'SELECT DISTINCT nome, texto FROM decisao_fts5 WHERE nome IN ({placeholders}) LIMIT ?', (*nomes, limite_linhas))
    rows = c.fetchall()
    conn.commit()
    result = {}
    for row in rows:
        nome, texto = row[0], row[1]
        index = texto.find(pesq_processada)
        if index != -1:
            start = max(0, index - tam_trecho)  # Evitar índice negativo
            end = min(index + len(query) + tam_trecho*2, len(texto))  # Ajustar para no sobrepasar el total de caracteres del texto
            trecho = '(...)' + texto[start:end] + '(...)'
        else:
            trecho = f'Há o termo {query} neste AIIM, mas não consegui separá-lo'
        # Substituindo todos os caracteres de controle ASCII por um espaço
        trecho = re.sub('[\x00-\x1F]', ' ', trecho)
        # Eliminar caracteres estranhos, con valores Unicode maiores que 512
        trecho = re.sub('[\u0200-\U0010FFFF]', '', trecho)
        result[nome] = trecho
    return result

# Exemplo de uso
resultado = pesquisa_trechos_aiims_proc(conn, c_mem, 'insanável', tam_trecho=50, limite_linhas=3)
for nome, trecho in resultado.items():
    print(f"{nome}: {trecho}")

Pesquisa: insanável
Pesquisa processada: insan
Dessa pesquisa processada, serão retornadas as seguintes palavras:
('insanáveis', 'insan')
('insanável', 'insan')
nomes: ['4125696voto4824801.txt', '4129468voto4826797.txt', '4119239voto4880095.txt', '4126281voto4823760.txt', '4122892voto4822390.txt', '4124356voto4821944.txt', '4120224voto4828892.txt', '4127611voto4845651.txt', '4121927voto4825058.txt', '4129492voto4841792.txt', '4120418voto4821938.txt', '4129981voto4850176.txt', '4125076voto4820878.txt', '4129096voto4828483.txt', '4121974voto4821103.txt', '4122289voto4819401.txt', '4122108voto4819688.txt', '4127395voto4828080.txt', '4127785voto4825662.txt', '4125356voto4822140.txt', '4124751voto4822259.txt', '4117207voto4821345.txt', '4120897voto4818193.txt', '4120383voto4818002.txt', '4120414voto4816710.txt', '4125914voto4822905.txt', '4117920voto4819154.txt', '4123843voto4821695.txt', '4126048voto4825049.txt', '4122242voto4820429.txt', '4120246voto4817498.txt', '4124766voto4822394.txt',

In [ ]:
# alguns testes adicionais mostrando o que funciona e o que não funciona, em rodadas CRISP, para aperfeiçoamento do código
# c_mem.execute('DELETE FROM palavras')
# texto_processado = processa_texto(c_mem, 'crédito creditamento credit creditou-se indevido indevida')
# c_mem.execute('SELECT palavra, palavra_proc FROM palavras LIMIT 20')
# rows = c_mem.fetchall()
# for row in rows:
#     print(row)